In [1]:
# Array and data functions
from glob import glob
from tqdm import tqdm
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

from MEDUSSA.measure import SizeDataFrame
from MEDUSSA.transform import ParamSampler, ConfIntSampler

In [3]:
BasePath = '/Users/reyesmatte/Priestia_megaterium_comparisons/Day1/' 

images = sorted(glob(f'{BasePath}*.tif'))

strains = np.unique([img.split('/')[-1].split('_')[1] for img in images])

dfs = []

for i in tqdm(range(len(strains))):

    strain = strains[i]
    
    masks = sorted(glob(f'{BasePath}*{strain}*.tif'))

    times = np.unique([int(mask.split('/')[-1].split('_')[2][1:]) for mask in masks])
    
    for t in times:
        
        segmentations = sorted(glob(f'{BasePath}*{strain}_t{t}_*.tif'))

        if t == 5:
            stage = 'Exponential'
        else:
            stage = 'Stationary'
        
        df = SizeDataFrame(segmentations,from_files=True)
        
        df.insert(0,'Strain',str(strain))
        df.insert(1,'Stage',stage)

        dfs.append(df)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:16<00:00,  2.40s/it]


In [5]:
BasePath = '/Users/reyesmatte/Priestia_megaterium_comparisons/Day2/' 

dfs_d2 = []

for i in tqdm(range(len(strains))):

    strain = strains[i]
    
    masks = sorted(glob(f'{BasePath}*{strain}*.tif'))

    times = np.unique([int(mask.split('/')[-1].split('_')[2][1:]) for mask in masks])
    
    for t in times:
        
        segmentations = sorted(glob(f'{BasePath}*{strain}_t{t}_*.tif'))

        if t == 5:
            stage = 'Exponential'
        else:
            stage = 'Stationary'
        
        df = SizeDataFrame(segmentations,from_files=True)
        
        df.insert(0,'Strain',str(strain))
        df.insert(1,'Stage',stage)

        dfs_d2.append(df)
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:17<00:00,  2.44s/it]


In [6]:
full_df = pd.concat(dfs,ignore_index=True)
full_df = full_df[full_df['Width'] > 0.3]

full_df_d2 = pd.concat(dfs_d2,ignore_index=True)
full_df_d2 = full_df_d2[full_df_d2['Width'] > 0.3]

metrics = ['Length','Width','SurfaceArea','Volume']
columns = ['Strain','Stage']

df_posteriors = pd.read_csv('https://raw.githubusercontent.com/OReyesMatte/MEDUSSA/refs/heads/main/Figures/FigureS4/posterior_segmentation_measurements.csv')

full_df = ParamSampler(full_df,df_posteriors,columns,metrics)
full_df.insert(0,'Day','Day1')

full_df_d2 = ParamSampler(full_df_d2,df_posteriors,columns,metrics)
full_df_d2.insert(0,'Day','Day2')

joint_df = pd.concat([full_df,full_df_d2],ignore_index=True)

joint_df.to_csv('joint_days_df.csv')

In [ ]:
strains = ['PY79','WH320','1060','7A242','QMB1551','DSM319','7A1']

stages = ['Exponential','Stationary']

strain_dict = {'PY79':'PY79','WH320':'WH320','1060':'DSM 32','7A242':'KM','QMB1551':'QM B1551','DSM319':'DSM 319','7A1':'899'}

colors = sns.color_palette('tab10')[:2]

handles = [Line2D([0], [0], color='w', lw=1, marker='o', markerfacecolor=c, markersize=10) for c in colors]

for strain in strains:

    for stage in stages:
        
        subdf = joint_df[joint_df['Strain']==strain]
        subdf = subdf[subdf['Stage']==stage]

        c1,c2 = subdf.groupby(['Day']).size().reset_index(name='counts')['counts']
        
        d1_w,d2_w = subdf[['Day','Width_mu']].groupby('Day').median()['Width_mu']
        d1_w_std,d2_w_std = subdf[['Day','Width_mu']].groupby('Day').std()['Width_mu']

        d1_L,d2_L = subdf[['Day','Length_mu']].groupby('Day').median()['Length_mu']
        d1_L_std,d2_L_std = subdf[['Day','Length_mu']].groupby('Day').std()['Length_mu']

        nl = '\n'
        
        label1 = f'Day 1,      W = {d1_w:.3f}, CV = {d1_w_std/d1_w:.3f}{nl}n = {c1}    L = {d1_L:.3f}, CV = {d1_L_std/d1_L:.3f}'
        label2 = f'Day 2,      W = {d2_w:.3f}, CV = {d2_w_std/d2_w:.3f}{nl}n = {c2}    L = {d2_L:.3f}, CV = {d2_L_std/d2_L:.3f}'

        fig,ax = plt.subplots(dpi=300,figsize=(4,4))

        sns.scatterplot(data=subdf,x='Width_mu',y='Length_mu',hue='Day',legend=True,alpha=0.5,ax=ax)
        ax.set_xlim(0.4,1.8)
        ax.set_ylim(0,25)
        ax.set_xlabel('Cell Width (µm)')
        ax.set_ylabel('Cell Length (µm)')
        ax.set_title(f'{strain_dict[strain]}, {stage}')
        ax.legend(handles=handles,labels=[label1,label2])

        fig.tight_layout()
        fig.savefig(f'/Users/reyesmatte/Desktop/Imaging_paper/FigSX/{strain}_{stage}.png')